### Libraries

In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.express as px
import scipy.stats as stats
from sklearn.model_selection import train_test_split, GridSearchCV
from dython.nominal import associations
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import ADASYN
from matplotlib import pyplot
from numpy import where
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso
import warnings
import pickle
from sklearn.dummy import DummyClassifier
from collections import Counter
import category_encoders as ce
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import TweedieRegressor
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import multiprocessing
import lightgbm as lgbm
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.dummy import DummyClassifier


from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 5000)

In [20]:
# nos guardamos la tabla a CSV
#df_data.to_csv('datos/df_data.csv', index=False)

In [21]:
#Leemos nuestra tabla de nuevo
df_coded = pd.read_csv('datos/df_coded.csv')
df_coded.head()

,V_YEAR,C_YEAR,P_AGE,C_WTHR_2,C_WTHR_3,C_WTHR_4,C_WTHR_5,C_WTHR_6,C_WTHR_7,C_WTHR_52,C_WTHR_55,C_RSUR_2,C_RSUR_3,C_RSUR_4,C_RSUR_5,C_RSUR_6,C_RSUR_7,C_RSUR_8,C_RSUR_9,C_RSUR_52,C_RSUR_55,C_RALN_2,C_RALN_3,C_RALN_4,C_RALN_5,C_RALN_6,C_RALN_52,C_RALN_55,P_SEX_1,P_SEX_8,P_SEX_9,P_SAFE_2,P_SAFE_9,P_SAFE_10,P_SAFE_11,P_SAFE_12,P_SAFE_13,P_SAFE_50,P_SAFE_54,P_SAFE_56,C_WDAY_2,C_WDAY_3,C_WDAY_4,C_WDAY_5,C_WDAY_6,C_WDAY_7,C_WDAY_52,V_TYPE,C_HOUR,TARGET,C_TRAF,C_RCFG,C_MNTH
0,-0.596056,-1.531636,-0.091281,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,6,20,0,3,50,1
1,-0.691561,-1.531636,-0.746470,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,20,0,3,50,1
2,-0.691561,-1.531636,-0.716688,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,20,0,3,50,1
3,-0.723396,-1.531636,0.057626,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,8,0,18,50,1
4,2.969452,-1.531636,-1.163408,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,54,8,0,18,50,1


### Dividimos el dataset en train y test
Dividimos el dataset en train y test manteniendo la proporción de la variable objetivo en las dos partes (separación estratificada)

In [12]:
X = df_coded.drop('TARGET',axis=1)
y = df_coded['TARGET']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, shuffle=True, test_size=0.20, random_state=0)

In [14]:
def evaluate_model(y_test, y_pred, y_pred_proba = None):
    if y_pred_proba is not None:
        print('ROC-AUC score of the model: {}'.format(roc_auc_score(y_test, y_pred_proba[:, 1])))
    print('Accuracy of the model: {}\n'.format(accuracy_score(y_test, y_pred)))
    print('Classification report: \n{}\n'.format(classification_report(y_test, y_pred)))
    print('Confusion matrix: \n{}\n'.format(confusion_matrix(y_test, y_pred)))

### Problema de desbalanceo de datos

Para solventar este problema probaremos a utilizar dos tecnicas entre muchas que existen para solucionar este problema: Borderline-SMOTE y Borderline-SMOTE SVM. Combinaremos ambas técnicas con los modelos para comprobar cual nos da mejores resultados segun el caso.

### Borderline-SMOTE
SMOTE realiza un oversample de la clase minoritaria segun la configuracion que le demos (strategy, k-values) pero estudios han demostrado que esta técnica obtiene mejores resultados si aplicamos la version Borderline que en vez de generar más población de la clase minoritaria de manera ciega, se centra en los puntos mas dificiles de clasificar debido a su cercanía con el grupo poblacional de la clase opuesta.

In [7]:
# borderline-SMOTE for imbalanced dataset
from collections import Counter
from imblearn.over_sampling import BorderlineSMOTE
# summarize class distribution
counter = Counter(y_train)
print(counter)
# transform the dataset
oversample = BorderlineSMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)
# summarize the new class distribution
counter = Counter(y_train)
print(counter)

Counter({0: 4642566, 1: 45758})
Counter({0: 4642566, 1: 4642566})


In [15]:
# simple-SMOTE for imbalanced dataset
from collections import Counter
from imblearn.over_sampling import SMOTE
# summarize class distribution
counter = Counter(y_train)
print(counter)
# transform the dataset
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)
# summarize the new class distribution
counter = Counter(y_train)
print(counter)

Counter({0: 4642566, 1: 45758})
Counter({0: 4642566, 1: 4642566})


### Escalado de las variables
Aunque en algunos salgoritmos no sea necesario el escalado de variables, pasaremos a realizarlo en este punto para homogeneizar el proceso.

In [9]:
scaler = StandardScaler()
model_scaled = scaler.fit(X_train)
X_train_scaled = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

NameError: name 'X_train' is not defined

### Modelos

In [34]:
dic = {
    'base': {
        'model': DummyClassifier(),
        'params': {
            'strategy': ['constant'],
            'constant': [1],
        }
    },

    'glm': {
        'model': LogisticRegression(),
        'params': {
            'solver': ['newton-cg', 'lbfgs', 'liblinear'],
            'penalty': ['l2'],
            'C': [100, 10, 1.0, 0.1, 0.01]
        }
    },
    
    'lss': {
        'model': Lasso(),
        'params': {
            'alpha': [0.1, 1, 10]
        }
    },
    
    'rf': {
        'model': RandomForestClassifier(),
        'params': {'bootstrap': [True, False],
                'max_depth': [10, 20, 30, 40, 50, None],
                'min_samples_leaf': [1, 2, 4, 8, 16]
        }
    },
    'nb': {
        'model': GaussianNB(),
        'params': {
            'var_smoothing': np.logspace(0,-9, num=5)
        }
    },
    
    
    'xgb': {
        'model': xgb.XGBClassifier(),
        'params': {
            'max_depth': [2, 4, 6],
            'n_estimators': [50, 100, 200]
        }     
    },
    
    'lgbm': {
        'model': lgbm.LGBMClassifier(),
        'params': {
            'objective' : ["regression"],
            'metric' : ["l2"],
            'min_data' : ['1L'],
            'learning_rate' : [1.0]
        }     
    }
}

In [ ]:
modelres = {}

for i in dic:
    res = {}
    
    cv = GridSearchCV(dic[i]['model'], dic[i]['params'], verbose=2)
    
    res['cv_model'] = cv
    
    cv.fit(X_train_scaled, y_train)

    res['best_params'] = cv.best_params_

    res['train_score'] = cv.score(X_train_scaled, y_train)

    res['test_score'] = cv.score(X_test_scaled, y_test)
    
    modelres[i] = res

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ......................constant=1, strategy=constant; total time=  13.0s
[CV] END ......................constant=1, strategy=constant; total time=   3.9s
[CV] END ......................constant=1, strategy=constant; total time=   3.6s
[CV] END ......................constant=1, strategy=constant; total time=   3.4s
[CV] END ......................constant=1, strategy=constant; total time=   3.4s
Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV] END ................C=100, penalty=l2, solver=newton-cg; total time= 4.5min
[CV] END ................C=100, penalty=l2, solver=newton-cg; total time= 5.6min
[CV] END ................C=100, penalty=l2, solver=newton-cg; total time= 5.1min
[CV] END ................C=100, penalty=l2, solver=newton-cg; total time= 5.5min
[CV] END ................C=100, penalty=l2, solver=newton-cg; total time= 5.0min
[CV] END ....................C=100, penalty=l2, solver=lbfgs; total ti

In [16]:
preprocessor= ColumnTransformer( transformers=[
('scaler', StandardScaler(), X_train.columns)])

In [17]:
classifiers = [
    DummyClassifier(strategy='constant', constant =1),
    Lasso(),
    LogisticRegression(), 
    RandomForestClassifier(),
    xgb.XGBClassifier(),
    lgbm.LGBMClassifier(),
    GaussianNB(),
    GradientBoostingClassifier()
]

In [18]:
for classifier in classifiers:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', classifier)]) 
    pipe.fit(X_train, y_train)
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))


DummyClassifier(constant=1, strategy='constant')
model score: 0.010
Lasso()
model score: -24.868
LogisticRegression()
model score: 0.796


In [22]:
classifiers = [
    DummyClassifier(strategy='constant', constant =1),
    Lasso(),
    LogisticRegression()
]

In [ ]:
for classifier in classifiers:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', classifier)]) 
    pipe.fit(X_train, y_train)
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))


DummyClassifier(constant=1, strategy='constant')
model score: 0.010
Lasso()
model score: -24.868
LogisticRegression()
model score: 0.796
